In [20]:
from typing import Dict
import openai
import os
!pip install openai textstat

from textstat import flesch_reading_ease

In [21]:
#  OpenAI API key from environment variable
OPENAI_API_KEY="generatedAPIkey"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Sample category-specific prompt templates
CATEGORY_PROMPTS = {
    "fashion": "Write a stylish and engaging product description for a {category}. Include material, color, style, usage occasions, and any unique features. Keep the tone elegant and modern.",
    "electronics": "Create a professional product description for a {category}. Emphasize specs, performance, compatibility, and user benefits. Tone should be informative and persuasive.",
    "home": "Write a clear and attractive description for a {category}. Highlight material, durability, size, benefits for cooking, and ease of cleaning. Tone should be friendly and helpful.",
    "books": "Write a compelling summary for a book titled '{title}' by {author}. Mention the genre, theme, and ideal reader. Tone should be enthusiastic and informative."
}


In [22]:
# Generic fallback prompt
GENERIC_PROMPT = "Write a short and engaging product description for a product in the {category} category. Highlight key features and ideal usage."

# Sanitize and enrich input
DEFAULTS = {
    "brand": "a trusted brand",
    "features": "great value and performance",
    "color": "various colors",
    "size": "standard size"
}

In [23]:
def sanitize_input(data: Dict) -> Dict:
    required = ['name', 'category']
    for key in required:
        if key not in data:
            raise ValueError(f"Missing required field: {key}")
    for key, default in DEFAULTS.items():
        data.setdefault(key, default)
    return data

In [24]:
# Prompt generator

def generate_prompt(data: Dict) -> str:
    category = data.get("category", "general").lower()
    if category in CATEGORY_PROMPTS:
        prompt_template = CATEGORY_PROMPTS[category]
    else:
        prompt_template = GENERIC_PROMPT
    return prompt_template.format(**data)



In [25]:
# OpenAI LLM call
# model="gpt-4",
def call_openai_llm(prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-1.0.0",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that writes high-quality e-commerce product descriptions."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response['choices'][0]['message']['content'].strip()



In [26]:
# Output evaluator
def evaluate_description(text: str) -> Dict:
    length = len(text.split())
    return {
        "readability_score": flesch_reading_ease(text),
        "word_count": length,
        "within_length_range": 60 <= length <= 120
    }



In [27]:
# Example run
def generate_product_description(data: Dict) -> Dict:
    data = sanitize_input(data)
    prompt = generate_prompt(data)
    output = call_openai_llm(prompt)
    evaluation = evaluate_description(output)
    return {
        "prompt": prompt,
        "description": output,
        "evaluation": evaluation
    }



In [ ]:
# Example usage
if __name__ == "__main__":
    sample_data = {
        "name": "Stylish Leather Bag",
        "category": "fashion",
        "material": "genuine leather",
        "color": "black",
        "style": "tote",
        "features": "gold zipper, inner pockets",
        "usage": "office, casual outings"
    }
    result = generate_product_description(sample_data)
    print("Prompt:\n", result["prompt"])
    print("\nGenerated Description:\n", result["description"])
    print("\nEvaluation:\n", result["evaluation"])
